In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0


In [27]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

/Users/ittichaiboonyarakthunya/Documents/WorkDir/developments/tensorflow_lab/.venv/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.719939  ,  0.10202616, -0.455853  , -0.02526802,  0.10255302,
         1.221045  , -0.38893792, -0.9512024 , -0.66902596,  0.46922708]],
      dtype=float32)

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.16507863, 0.08898868, 0.05093906, 0.07835227, 0.08903557,
        0.27246985, 0.05446427, 0.03104012, 0.0411596 , 0.12847188]],
      dtype=float32)

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()

np.float32(1.3002273)

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 605us/step - accuracy: 0.8525 - loss: 0.5007
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 574us/step - accuracy: 0.9542 - loss: 0.1524
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 674us/step - accuracy: 0.9676 - loss: 0.1080
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 547us/step - accuracy: 0.9730 - loss: 0.0848
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 550us/step - accuracy: 0.9778 - loss: 0.0726


In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - 476us/step - accuracy: 0.9765 - loss: 0.0717


[0.07172075659036636, 0.9764999747276306]

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.51148464e-07, 3.39249340e-08, 5.48411481e-06, 1.23405887e-03,
        8.68722455e-12, 4.46819627e-07, 7.64658044e-14, 9.98751581e-01,
        6.86421345e-07, 7.49969922e-06],
       [3.07403418e-08, 1.88109916e-04, 9.99776304e-01, 2.94203164e-05,
        2.50889787e-14, 3.74131162e-08, 1.25108968e-09, 1.07800684e-13,
        6.02129057e-06, 2.88024406e-13],
       [4.40794838e-06, 9.99257088e-01, 8.57992054e-05, 8.58210115e-06,
        4.43783538e-05, 1.77303764e-06, 2.27189194e-05, 3.56574106e-04,
        2.17001507e-04, 1.72719842e-06],
       [9.99799311e-01, 2.82568529e-08, 6.71894959e-05, 6.57864632e-07,
        1.06976438e-06, 7.12887640e-06, 8.87098577e-05, 2.56694748e-05,
        4.11474673e-07, 9.84927374e-06],
       [1.08274230e-06, 4.09114687e-10, 9.95546816e-06, 6.51634053e-08,
        9.57645178e-01, 1.99600720e-07, 4.71019348e-06, 1.66467813e-04,
        1.24711562e-06, 4.21711616e-02]], dtype=float32)>

### Advanced Step

In [35]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [36]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [37]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

In [38]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [39]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [40]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [41]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [44]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_state()
  train_accuracy.reset_state()
  test_loss.reset_state()
  test_accuracy.reset_state()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

2025-03-03 09:40:39.598852: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-03-03 09:40:40.067687: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1, Loss: 0.14482250809669495, Accuracy: 95.6449966430664, Test Loss: 0.06482434272766113, Test Accuracy: 97.82999420166016


2025-03-03 09:40:57.042124: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2, Loss: 0.045262571424245834, Accuracy: 98.57333374023438, Test Loss: 0.06706319749355316, Test Accuracy: 97.66999816894531
Epoch 3, Loss: 0.024623384699225426, Accuracy: 99.19999694824219, Test Loss: 0.05718202516436577, Test Accuracy: 98.18999481201172


2025-03-03 09:41:34.603164: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 4, Loss: 0.015023390762507915, Accuracy: 99.5199966430664, Test Loss: 0.062097251415252686, Test Accuracy: 98.18999481201172
Epoch 5, Loss: 0.01030388381332159, Accuracy: 99.62833404541016, Test Loss: 0.06814456731081009, Test Accuracy: 98.15999603271484
